In [43]:
import sys
import os
import pandas as pd
from pathlib import Path


In [44]:
# Añadir el directorio src al path de Python
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)


In [45]:
from calculadora_margen.encoder import Encoder
from calculadora_margen.cleaning.cleaner_df import DataFrameCleaner
from calculadora_margen.cleaning.params import Parameters
from calculadora_margen.cleaning.validador import Validator
from calculadora_margen.cleaning.outliers_manager import OutliersManager

In [46]:
project_root_path = Path(src_path).parent
data_path = project_root_path / 'data'
raw_path = data_path / 'raw'
clean_path = data_path / 'clean'

ETL master_lotes

In [47]:
master_lotes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [48]:
cleaner = DataFrameCleaner(master_lotes)
params = Parameters.master_lotes

master_lotes = (cleaner
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .rows_cleaner.drop_duplicates()
    .rows_cleaner.drop_na(params.drop_na_subset)
    .data_cleaner.to_upper()
    .get_df()
)

In [49]:
validator = Validator(master_lotes)
master_lotes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

In [50]:
# Creamos clave única para poder hacer merge en otros df
encoder = Encoder(master_lotes)
master_lotes = encoder.create_key(col1='articulo', col2='lote_proveedor', new_col_name='clave_merge')

In [51]:
duplicados = master_lotes['clave_merge'].duplicated().sum()
duplicados

np.int64(0)

In [52]:
master_lotes.sample(2)

,articulo,lote_proveedor,lote_componente,clave_merge
5780,FCAR046,160424,160424,FCAR046-160424
16673,SCRE061,250124,250124,SCRE061-250124


In [53]:
master_lotes.to_csv(clean_path / 'master_lotes_clean.csv', index=False)

ETL costes

In [54]:
costes = pd.read_csv(raw_path / 'costes.csv',  encoding='UTF-8', sep=';', dtype=str)

In [55]:
cleaner = DataFrameCleaner(costes)
params = Parameters.costes

costes = (cleaner
    .rows_cleaner.drop_na(params.drop_na_subset)
    .rows_cleaner.drop_duplicates()
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .data_cleaner.fix_numeric_format(params.cols_to_float)
    .rows_cleaner.drop_duplicates_batch(params.drop_duplicates_subset)
    .data_cleaner.to_upper()
    .get_df()
)

In [56]:
validator = Validator(costes)

costes = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

Tamaño inicial del DataFrame: 8000

Validando columna: componente
Filas inválidas encontradas: 1

Validando columna: lote_componente
Filas inválidas encontradas: 21

Tamaño final del DataFrame: 7978


In [57]:
# Ver las filas inválidas para una columna específica
#invalid_rows = validator.get_invalid('lote_interno')
#print(invalid_rows.head(10))

In [58]:
outliers_manager = OutliersManager(costes)

costes = (outliers_manager
    .process_outliers()
    .clean_columns()
    .get_df()
)


=== RESUMEN DE OUTLIERS ===
Outliers detectados inicialmente: 71
Outliers reemplazados por la media: 66
Outliers restantes: 5

Detalle de outliers restantes:
- MAT101: 1 outliers (desviación media: 0.0%)
- MAT183: 3 outliers (desviación media: 0.0%)
- MAT265: 1 outliers (desviación media: 0.0%)


In [59]:
costes.to_csv(clean_path / 'costes_clean.csv', index=False)

In [60]:
costes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7978 entries, 0 to 7977
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   coste_componente_unitario  7978 non-null   float64
 1   lote_componente            7978 non-null   object 
 2   componente                 7978 non-null   object 
dtypes: float64(1), object(2)
memory usage: 187.1+ KB


ETL fabricaciones

In [61]:
fabricaciones = pd.read_csv(raw_path / 'fabricaciones_2025.csv',  encoding='UTF-8', sep=';', dtype=str)

In [62]:
# No queremos trabajar con lote_componente_proveedor, unimos a master_lotes para obtener lote_componente
encoder = Encoder(fabricaciones)

fabricaciones = encoder.create_key(col1='Componente', col2='Lote Componente', new_col_name='clave_merge')
fabricaciones = fabricaciones.merge(master_lotes, on="clave_merge", how="left")

In [63]:
cleaner = DataFrameCleaner(fabricaciones)
params = Parameters.fabricaciones

fabricaciones = (cleaner
    .rows_cleaner.drop_duplicates()
    .columns_cleaner.keep_and_rename(params.cols_to_keep, params.rename_map)
    .data_cleaner.fix_numeric_format(params.cols_to_float)
    .rows_cleaner.drop_na(params.drop_na_subset)
    .data_cleaner.to_upper()
    .get_df()
)

In [64]:
validator = Validator(fabricaciones)
fabricaciones = (validator
    .validate_with_map(params.validation_map)
    .get_df()
)

Tamaño inicial del DataFrame: 19583

Validando columna: articulo
Filas inválidas encontradas: 0

Validando columna: componente
Filas inválidas encontradas: 0

Tamaño final del DataFrame: 19583


In [65]:
fabricaciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19583 entries, 0 to 19658
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   fecha_fabricacion    19583 non-null  object 
 1   articulo             19583 non-null  object 
 2   lote_articulo        19583 non-null  object 
 3   unidades_fabricadas  19583 non-null  float64
 4   componente           19583 non-null  object 
 5   lote_componente      19583 non-null  object 
 6   consumo_unitario     19583 non-null  float64
 7   consumo_total        19583 non-null  float64
 8   id_orden             19583 non-null  object 
dtypes: float64(3), object(6)
memory usage: 1.5+ MB


In [66]:
fabricaciones.to_csv(clean_path / 'fabricaciones_clean.csv', index=False)

In [67]:
master_articulos = pd.read_csv(raw_path / 'master_articulos.csv',  encoding='UTF-8', sep=';', dtype=str)